In [1]:
import polars as pl
import numpy as np
import requests

In [2]:
project_df = pl.from_records(requests.get('https://api.emsl.pnl.gov/nexus/elasticsearch/projects/').json())

In [7]:
project_df.columns

['uuid',
 'id',
 'status',
 'title',
 'abstract',
 'pis',
 'institution_names',
 'active_members',
 'science_areas',
 'project_type',
 'award_doi',
 'accepted_date',
 'started_date',
 'closed_date',
 'estimated_end_date']

In [16]:
project_trimmed_df = project_df[['active_members', 'started_date', 'title', 'award_doi', 'id', 'uuid', 'science_areas', 'project_type']]

In [ ]:
project_trimmed_df = project_trimmed_df.with_columns(("https://sc-data.emsl.pnnl.gov/?projectId=" + project_trimmed_df['id']).alias('url'))

In [17]:
project_trimmed_df = project_trimmed_df.rename({'title': 'name', 'started_date': 'created_date', 'uuid': 'proposal_identifier', 'award_doi': 'citation_info', 'active_members': 'associated_researchers'})

In [9]:
project_trimmed_df.write_json("emsl_projects.json")

In [19]:
project_dicts = project_trimmed_df.to_dicts()

In [21]:
# project_dicts[0]
reformatted_projects = []
for project in project_dicts:
    reformatted_project = project
    reformatted_project['metadata'] = {'science_area': project['science_areas'][0], 'project_type': project['project_type']}
    reformatted_project.pop('science_areas')
    reformatted_project.pop('project_type')
    reformatted_projects.append(reformatted_project)

In [26]:
import json
with open('emsl_datasets.json', 'w') as f:
    f.write(json.dumps(reformatted_projects))

In [27]:
pl.from_dicts(reformatted_projects).schema

Schema([('associated_researchers',
         List(Struct({'id': Int64, 'name': String, 'orcid': String, 'institution': String, 'project_role': String}))),
        ('created_date', String),
        ('name', String),
        ('citation_info', String),
        ('id', String),
        ('proposal_identifier', String),
        ('url', String),
        ('metadata',
         Struct({'science_area': String, 'project_type': String}))])